<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Two: Exploratory data analysis and feature extraction. </h2>	


<h5> In this section, Compute at least 15 such image features (a method for each), including the following (NOTE: At least 10 of these must be scalar features and 2 matrix-based features): (i) image size, (ii) average of the red-channel intensity, (iii) aspectratio. This will require significant explatoratory research and data analysis. The first one is already implemented for you, and the next two are pre-specified. Additional requirements specfied in pdf. </h5>

In [1]:
%store -r train
%store -r test

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from numpy.linalg import matrix_rank
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd

In [3]:
shape1 = []
for i in range(train.shape[0]):
    shape1.append(train['Pictures'][i].shape[2])
np.mean(shape1)

3.0

In [4]:
# Returns the pixel size of the image
def ft0(image):
    return [image.size]

#Returns the aspect ratio of the image
def ft1(image):
    return [image.shape[0] / image.shape[1]]

def resize(image):
    return cv2.resize(image, (400, 340))

# Returns the average of the red-channel pictures for the images
def ft1(image):
    return [np.mean(image, axis=(0, 1))[0]]

#Returns the aspect ratio of the image
def ft2(image):
    return [np.mean(image, axis=(0, 1))[2]]

# Returns the average of the green-channel pictures for the images
def ft3(image):
    return [np.mean(image, axis=(0, 1))[0]]

# Returns the average of the blue-channel pictures for the images
def ft4(image):
    return [np.mean(image, axis=(0, 1))[1]]
    
# Returns the cornerHarris proportion
def ft5(image):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)
    return [(dst>0.01 * dst.max()).sum() / dst.size]

# Returns the cornerHarris rank divided by max possible rank
def ft6(image):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)
    return [matrix_rank(dst) / min(dst.shape)]

# Returns the X Y coordinate of the first 5 SIFT features
def ft7(image):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create(nfeatures=5)
    kps, descs = sift.detectAndCompute(gray, None)
    pts = []
    for i in range(len(kps)):
        pts.append(kps[i].pt)
    res = np.array(pts).flatten()
    return np.pad(res, (0, max(0, 10 - len(res))))
    

# Returns the descriptors of the first 5 SIFT features
def ft8(image):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create(nfeatures=5)
    kps, descs = sift.detectAndCompute(gray, None)
    if len(kps) == 0:
        return np.zeros(10)
    res = descs.flatten()
    return np.pad(res, (0, max(0, 10 - len(res))))

# Returns the X Y of the best 5 features to track
def ft9(image):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    corners = cv2.goodFeaturesToTrack(gray, 5, 0.01, 10)
    corners = np.int0(corners)
    res = corners.ravel()
    return np.pad(res, (0, max(0, 10 - len(res))))

# Returns the X Y coordinate of the best 5 ORB features
def ft10(image):
    orb  = cv2.ORB_create(nfeatures=5)
    kps, descs = orb.detectAndCompute(image, None)
    pts = []
    for i in range(len(kps)):
        pts.append(kps[i].pt)
    res = np.array(pts).flatten()
    return np.pad(res, (0, max(0, 10 - len(res))))

# Returns the descriptors of the best 5 ORB features
def ft11(image):
    orb  = cv2.ORB_create(nfeatures=5)
    kps, descs = orb.detectAndCompute(image, None)
    if len(kps) == 0:
        return np.zeros(10)
    res = descs.flatten()
    return np.pad(res, (0, max(0, 10 - len(res))))

 We expect all external sources sited, and significant indication of EDA. 

<h4> Graphs </h4>

In [51]:
img = train['Pictures'][222]
img = resize(img)
orb  = cv2.ORB_create(nfeatures=5)
kps, descs = orb.detectAndCompute(img, None)
len(kps)

1

<h4> Sources </h4>

<h4> DataFrame Creation </h4>

In [16]:
def feature_frame(df):
    X = pd.DataFrame()
    pictures = df['Pictures']
    for idx, img in enumerate(pictures):
        if idx % 10 == 0:
            print(idx)
        row = []
        row.extend(ft0(img))
        row.extend(ft1(img))
        img = resize(img)
        row.extend(ft2(img))
        row.extend(ft3(img))
        row.extend(ft4(img))
        row.extend(ft5(img))
        row.extend(ft6(img))
        row.extend(ft7(img))
        row.extend(ft8(img))
        row.extend(ft9(img))
        row.extend(ft10(img))
        row.extend(ft11(img))
        series = pd.Series(row)
        X = X.append(series, ignore_index=True)
    return X

In [44]:
ft11(train.loc[222,'Pictures']).shape[0]
ft11(train.loc[223,'Pictures']).shape[0]

160

In [35]:
feature_frame(train.loc[222:223,:])

0


,0,1,2,3,4,5,6,7,8,9,...,699,700,701,702,703,704,705,706,707,708
0,538389.0,58.755515,65.793353,58.646816,60.742338,0.001868,0.097059,2.575972,328.519592,212.967575,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,391389.0,28.762423,37.363463,28.642684,31.743316,0.001868,0.097059,2.575972,328.519592,212.967575,...,237.0,210.0,172.0,221.0,39.0,246.0,86.0,228.0,70.0,241.0


In [101]:
X_train = feature_frame(train)
y_train = train['Encoding']

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500


In [102]:
%store X_train
%store y_train

Stored 'X_train' (DataFrame)
Stored 'y_train' (Series)
